# Faster R-CNN 구현
### 텐서플로우 기반
### 주석은 한글 위주로 작성

In [1]:
import numpy as np
import os
import glob
import cv2
import xmltodict
import tensorflow as tf
import math
import time
import random
from tensorflow.keras.layers import Lambda
from tqdm import tqdm
from PIL import Image, ImageDraw
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)


## 훈련 이미지 가져오기

In [2]:
train_x_path = '/Users/minguinho/Documents/AI_Datasets/PASCAL_VOC_2007/train/VOCdevkit/VOC2007/JPEGImages'
train_y_path = '/Users/minguinho/Documents/AI_Datasets/PASCAL_VOC_2007/train/VOCdevkit/VOC2007/Annotations'

test_x_path = '/Users/minguinho/Documents/AI_Datasets/PASCAL_VOC_2007/test/VOCdevkit/VOC2007/JPEGImages'
test_y_path = '/Users/minguinho/Documents/AI_Datasets/PASCAL_VOC_2007/test/VOCdevkit/VOC2007/Annotations'

In [3]:
list_train_x = sorted([x for x in glob.glob(train_x_path + '/**')])    
list_train_y = sorted([x for x in glob.glob(train_y_path + '/**')]) 

list_test_x = sorted([x for x in glob.glob(test_x_path + '/**')])    
list_test_y = sorted([x for x in glob.glob(test_y_path + '/**')]) 

print(len(list_train_x))
print(len(list_test_x))

5011
4952


## 훈련용 데이터로 제작

In [4]:
# 앵커 테스트용
def get_image_224(image_file_path) :
    image = cv2.imread(image_file_path)
    image = cv2.resize(image, (224, 224))

    return image

In [5]:
# 입력용 이미지 생성. 224, 224로 변환시키고 채널 값(0~255)를 0~1 사이의 값으로 정규화 시켜줌
def make_input(image_file_list): 
    images_list = []
    
    for i in tqdm(range(0, len(image_file_list)), desc="get image") :
        
        image = cv2.imread(image_file_list[i])
        image = cv2.resize(image, (224, 224))/255
        
        images_list.append(image)
    
    return np.asarray(images_list)

In [6]:
# RPN 훈련할 때는 객체 종류를 알 필요가 없다. 왜냐하면 RPN은 객체가 존재하는 박스 위치만 알면 되기 때문이다. 
# 그러면 객체 종류를 어디서 알아야 하는 것일까. 바로 Detector를 훈련시킬 때 필요하다. 
# 존재하는 객체 종류를 알아내자
def get_Classes_inImage(xml_file_list):
    classes = []

    for xml_file_path in xml_file_list: 

        f = open(xml_file_path)
        xml_file = xmltodict.parse(f.read())
        # 사진에 객체가 여러개 있을 경우
        try: 
            for obj in xml_file['annotation']['object']:
                classes.append(obj['name'].lower()) # 들어있는 객체 종류를 알아낸다
        # 사진에 객체가 하나만 있을 경우
        except TypeError as e: 
            classes.append(xml_file['annotation']['object']['name'].lower()) 
        f.close()

    classes = list(set(classes)) # set은 중복된걸 다 제거하고 유니크한? 아무튼 하나만 가져온다. 그걸 리스트로 만든다
    classes.sort() # 정렬

    return classes

## RPN 훈련을 위한 데이터셋 생성에 필요한 함수

In [7]:
# 이미지에 어떤 Ground Truth Box가 있는지
def get_label_fromImage_RPN(xml_file_path): # xml_file_path은 파일 하나의 경로를 나타낸다

    f = open(xml_file_path)
    xml_file = xmltodict.parse(f.read()) 

    # 우선 원래 이미지 크기를 얻는다. 왜냐하면 앵커는 224*224 기준으로 만들었는데 원본 이미지는 224*224가 아니기 때문.
    # 224*224에 맞게 줄일려고 하는거다
    Image_Height = float(xml_file['annotation']['size']['height'])
    Image_Width  = float(xml_file['annotation']['size']['width'])

    Ground_Truth_Box_list = [] 

    # multi-objects in image
    try:
        for obj in xml_file['annotation']['object']:
            
            # 박스 좌표(왼쪽 위, 오른쪽 아래) 얻기
            x_min = float(obj['bndbox']['xmin']) 
            y_min = float(obj['bndbox']['ymin'])
            x_max = float(obj['bndbox']['xmax']) 
            y_max = float(obj['bndbox']['ymax'])

            # 224*224에 맞게 변형시켜줌
            x_min = float((224/Image_Width)*x_min)
            y_min = float((224/Image_Height)*y_min)
            x_max = float((224/Image_Width)*x_max)
            y_max = float((224/Image_Height)*y_max)

            Ground_Truth_Box = [x_min, y_min, x_max, y_max]
            Ground_Truth_Box_list.append(Ground_Truth_Box)

    # single-object in image
    except TypeError as e : 
        # 박스 좌표(왼쪽 위, 오른쪽 아래) 얻기
        x_min = float(xml_file['annotation']['object']['bndbox']['xmin']) 
        y_min = float(xml_file['annotation']['object']['bndbox']['ymin']) 
        x_max = float(xml_file['annotation']['object']['bndbox']['xmax']) 
        y_max = float(xml_file['annotation']['object']['bndbox']['ymax']) 

        # 224*224에 맞게 변형시켜줌
        x_min = float((224/Image_Width)*x_min)
        y_min = float((224/Image_Height)*y_min)
        x_max = float((224/Image_Width)*x_max)
        y_max = float((224/Image_Height)*y_max)

        Ground_Truth_Box = [x_min, y_min, x_max, y_max]  
        Ground_Truth_Box_list.append(Ground_Truth_Box)

    
    Ground_Truth_Box_list = np.asarray(Ground_Truth_Box_list)
    Ground_Truth_Box_list = np.reshape(Ground_Truth_Box_list, (-1, 4))

    return Ground_Truth_Box_list # 이미지에 있는 Ground Truth Box 리스트 받기(numpy)


## RPN - 앵커 준비
#### 입력 이미지 기준으로 앵커를 생성한다.
#### 풀링을 3번 하므로 2^3 = 8이니 (8, 8)부터 (16, 8)...등 8픽셀식 중심 좌표를 옮겨가며 앵커들을 k개씩 생성한다
#### 생성한 앵커들 중 사용할 가치가 있는 앵커를 걸러낸다(이미지 범위를 벗어나지 않는 앵커들만 선정)
#### 선정한 앵커 중 실제 물체의 box와 얼마나 곂치는지(IoU) 계산해본다. 확실히 겹친다 하는 애들을 Positive 앵커로, 거의 안겹친다 하는 애들은 Negataive 앵커로 선정한다. 애매한 애들은 거른다.
#### 이렇게 생성된 앵커들로 미니배치를 생성한다. Positive 128개, Negative 128개로 만드는게 ideal한 구성이긴 한데 Positive한 앵커가 별로 없다. 그래서 Positive를 128개 못채웠으면 Negataive 앵커로 채워준다. 

In [8]:
# 앵커 생성 함수. 
def make_anchor(anchor_size, anchor_aspect_ratio) :
    # 입력 이미지(그래봤자 224*224긴 하지만)에 맞춰 앵커를 생성해보자 

    anchors = [] # [x,y,w,h]로 이루어진 리스트 
    anchors_state = [] # 이 앵커를 훈련에 쓸건가? 각 앵커별로 사용 여부를 나타낸다. 

    # 앵커 중심좌표 간격
    interval_x = 16
    interval_y = 16
    Center_max_x = 208 # 224 - 16, 중심좌표가 224가 될 수는 없다.
    Center_max_y = 208 # 224 - 16

    # 2단 while문 생성
    x = 8
    y = 8
    index_count = 0
    while(y <= 224): # 8~208 = 14개 
        while(x <= 224): # 8~208 = 14개 
            # k개의 앵커 생성. 여기서 k = len(anchor_size) * len(anchor_aspect_ratio)다
            for i in range(0, len(anchor_size)) : 
                for j in range(0, len(anchor_aspect_ratio)) :
                    anchor_width = anchor_aspect_ratio[j][0] * anchor_size[i]
                    anchor_height = anchor_aspect_ratio[j][1] * anchor_size[i]

                    anchor = [x, y, anchor_width, anchor_height]
                    anchors.append(anchor)
                    # 앵커가 이미지 경계선을 넘나드나? 필터링
                    if((x - (anchor_width/2) >= 0) and (y - (anchor_height/2) >= 0) and
                    (x + (anchor_width/2) <= 224) and (y + (anchor_height/2) <= 224)):
                        # 경계 안에 있으면 1
                        anchors_state.append(int(1))
                    else :
                        anchors_state.append(int(0))
            x = x + interval_x 
        y = y + interval_y
        x = 8
    return np.asarray(anchors), np.asarray(anchors_state) # 넘파이로 반환

### IoU 계산 후 Positive, Negative 앵커 분류

In [9]:
# 앵커들을 Positive, Negative 앵커로 나누고 각 앵커가 참고한 Ground Truth Box와 Class를 반환하자
# RPN에는 '어떤 클래스인가?'는 알 필요가 없다. '객체인가 아닌가'이거 하나만 필요할 뿐. 
def align_anchor(anchors, anchors_state, Ground_Truth_Box_list):

    # 각 앵커는 해당 위치에서 구한 여러가지 Ground truth Box와의 ioU 중 제일 높은거만 가져온다. 
    IoU_List = np.array([])
    Ground_truth_box_Highest_IoU_List = [] # 각 앵커가 어떤 Ground Truth Box를 보고 IoU를 계산했는가?

    #start = time.time()

    for i in range(0, len(anchors)):
        if anchors_state[i] == 0 :
            IoU_List = np.append(IoU_List, 0)
            Ground_truth_box_Highest_IoU_List.append([0,0,0,0])

            if i % 9 == 8 :
                IoU_List_inOneSpot = IoU_List[i-8:i+1]
                for num in list(range(i-8, i + 1)):
                    if IoU_List[num] > 0.7 or (max(IoU_List_inOneSpot) == IoU_List[num] and IoU_List[num] >= 0.3): # positive anchor
                        anchors_state[num] = 2
                    elif IoU_List[num] < 0.3 : # negative anchor
                        anchors_state[num] = 1
                    else: # 애매한 앵커들
                        anchors_state[num] = 0    
        else:
            anchor_minX = anchors[i][0] - (anchors[i][2]/2)
            anchor_minY = anchors[i][1] - (anchors[i][3]/2)
            anchor_maxX = anchors[i][0] + (anchors[i][2]/2)
            anchor_maxY = anchors[i][1] + (anchors[i][3]/2)

            anchor = [anchor_minX, anchor_minY, anchor_maxX, anchor_maxY]

            # 연산 속도 때문에 Box대신 Ground Truth Box의 인덱스를 저장
            IoU_max = 0
            ground_truth_box_Highest_IoU = [0,0,0,0]

            for j in range(0, len(Ground_Truth_Box_list)):

                ground_truth_box = Ground_Truth_Box_list[j]

                InterSection_min_x = max(anchor[0], ground_truth_box[0])
                InterSection_min_y = max(anchor[1], ground_truth_box[1])

                InterSection_max_x = min(anchor[2], ground_truth_box[2])
                InterSection_max_y = min(anchor[3], ground_truth_box[3])

                InterSection_Area = 0

                if (InterSection_max_x - InterSection_min_x + 1) >= 0 and (InterSection_max_y - InterSection_min_y + 1) >= 0 :
                    InterSection_Area = (InterSection_max_x - InterSection_min_x + 1) * (InterSection_max_y - InterSection_min_y + 1)

                box1_area = (anchor[2] - anchor[0]) * (anchor[3] - anchor[1])
                box2_area = (ground_truth_box[2] - ground_truth_box[0]) * (ground_truth_box[3] - ground_truth_box[1])
                Union_Area = box1_area + box2_area - InterSection_Area

                IoU = (InterSection_Area/Union_Area)
                if IoU > IoU_max :
                    IoU_max = IoU
                    ground_truth_box_Highest_IoU = ground_truth_box

            IoU_List = np.append(IoU_List, IoU_max)
            Ground_truth_box_Highest_IoU_List.append(ground_truth_box_Highest_IoU)

            # 한 위치에 9개의 앵커 존재 -> 9개 앵커에 대한 IoU를 계산할 때마다 모아서 Positive, Negative 앵커 분류
            if i % 9 == 8 :
                IoU_List_inOneSpot = IoU_List[i-8:i+1]
                for num in list(range(i-8, i + 1)):
                    if IoU_List[num] > 0.7 or (max(IoU_List_inOneSpot) == IoU_List[num] and IoU_List[num] >= 0.3): # positive anchor
                        anchors_state[num] = 2
                    elif IoU_List[num] < 0.3 : # negative anchor
                        anchors_state[num] = 1
                    else: # 애매한 앵커들
                        anchors_state[num] = 0     

    Ground_truth_box_Highest_IoU_List = np.asarray(Ground_truth_box_Highest_IoU_List)
    Ground_truth_box_Highest_IoU_List = np.reshape(Ground_truth_box_Highest_IoU_List, (-1, 4))
            
    return anchors_state, Ground_truth_box_Highest_IoU_List # 각 앵커의 상태, (모든)앵커가 IoU 계산에 참조한 Ground Truth Box

## RPN을 위한 데이터셋 생성

In [10]:
# RPN훈련을 위한 데이터셋. 
# RPN과 Detector는 별개의 모델이다. 즉, 두 모델을 훈련시킬 때 필요한 데이터셋은 따로따로 만들어야한다. 
# 여기선 RPN 훈련에 필요한 데이터만 만들거다. 
def make_dataset_forRPN(input_list) :
    image_file_list = input_list[0]
    xml_file_list = input_list[1]
    anchors = input_list[2]
    anchors_state = input_list[3]

    image_list = make_input(image_file_list) # 입력
    # 출력
    cls_layer_label_list = np.array([])
    reg_layer_label_list = np.array([])

    # 값 계속 생성하는거 막기위한 변수
    cls_label_forPositive = np.array([1.0,0.0])
    cls_label_forNegative = np.array([0.0,1.0])
    cls_label_forUseless  = np.array([0.0,0.0])

    reg_label_forNotPositive = np.array([0.0, 0.0, 0.0, 0.0])

    for i in tqdm(range(0, len(xml_file_list)), desc="get label"): # 각 이미지별로 데이터셋 생성(5011개)

        anchors_state_for = anchors_state # anchors_state는 매 사진마다 다르니까 원본값(?)을 복사해서 쓴다. 
        Ground_Truth_Box_list = get_label_fromImage_RPN(xml_file_list[i]) # 여기서는 Ground Truth Box에 대한 정보만 필요하다
        anchors_state_for, Ground_truth_box_Highest_IoU_List = align_anchor(anchors, anchors_state_for, Ground_Truth_Box_list)
        # 어떤 앵커가 Pos, neg 앵커인지, (모든)앵커가 참조한 ground truth box는 뭔지
    
        #start = time.time()
        # 연산 시간 때문에 생성(1761개치 모아놨다가 한 번에 추가하기)
        cls_layer_label_list_for = np.array([])
        reg_layer_label_list_for = np.array([])
        for j in range(0, len(anchors_state_for)) :
            if anchors_state_for[j] == 2 : # positive
                Ground_truth_box = np.array([Ground_truth_box_Highest_IoU_List[j][0] + Ground_truth_box_Highest_IoU_List[j][2]/2, Ground_truth_box_Highest_IoU_List[j][1] + Ground_truth_box_Highest_IoU_List[j][2]/2, Ground_truth_box_Highest_IoU_List[j][2] - Ground_truth_box_Highest_IoU_List[j][0], Ground_truth_box_Highest_IoU_List[j][3] - Ground_truth_box_Highest_IoU_List[j][1]])
                cls_layer_label_list_for = np.append(cls_layer_label_list_for, cls_label_forPositive)
                reg_layer_label_list_for = np.append(reg_layer_label_list_for, Ground_truth_box) # IoU계산에 참조한(pos, neg 분류에 기여한) Ground Truth Box의 정보 휙득
            elif anchors_state_for[j] == 1 : # negative는 Ground Truth Box 정보가 필요없으니 [0,0,0,0]을 넣는다. 
                cls_layer_label_list_for = np.append(cls_layer_label_list_for, cls_label_forNegative) # 해당 앵커 output이 [0,1] -> negative
                reg_layer_label_list_for = np.append(reg_layer_label_list_for, reg_label_forNotPositive)
            else : 
                cls_layer_label_list_for = np.append(cls_layer_label_list_for, cls_label_forUseless) # 해당 앵커 output이 [0.5, 0.5] -> 무의미한 값
                reg_layer_label_list_for = np.append(reg_layer_label_list_for, reg_label_forNotPositive)
        
        # 넘파이 배열로 변환
        cls_layer_label_list = np.append(cls_layer_label_list, cls_layer_label_list_for)
        reg_layer_label_list = np.append(reg_layer_label_list, reg_layer_label_list_for)
        #print("\nmaking label time :", time.time() - start)

    # 논문에서 말한 출력값 크기에 맞게 reshape
    cls_layer_label_list = np.reshape(cls_layer_label_list, (-1, 1764, 2)) 
    reg_layer_label_list = np.reshape(reg_layer_label_list, (-1, 1764, 4))

    image_list = image_list.astype('float32')

    return image_list, cls_layer_label_list, reg_layer_label_list # 훈련 데이터들(입, 출력)

## RPN에 쓰일 데이터셋 생성

In [11]:
# del image_file_list, xml_file_list, anchor_size, anchor_aspect_ratio, anchors, anchors_state

In [11]:
# 파일 리스트
image_file_list = sorted([x for x in glob.glob(train_x_path + '/**')])
xml_file_list = sorted([x for x in glob.glob(train_y_path + '/**')])

anchor_size = [32, 64, 128] # 이미지 크기가 224*224라 32, 64, 128로 지정
anchor_aspect_ratio = [[1,1],[1,0.5], [0.5,1]] # W*L기준 
anchors, anchors_state = make_anchor(anchor_size, anchor_aspect_ratio) # 앵커 생성 + 유효한 앵커 인덱스 휙득

image_list, cls_layer_label_list, reg_layer_label_list = make_dataset_forRPN([image_file_list, xml_file_list, anchors, anchors_state])

get label: 100%|██████████| 5011/5011 [15:54<00:00,  5.25it/s]


In [12]:
cls_layer_label_list.shape

(5011, 1764, 2)

In [12]:
print("image_list.shape : ", image_list.shape)
print("cls_layer_label_list.shape : ", cls_layer_label_list.shape)
print("reg_layer_label_list.shape : ", reg_layer_label_list.shape)

image_list.shape :  (5011, 224, 224, 3)
cls_layer_label_list.shape :  (5011, 1764, 2)
reg_layer_label_list.shape :  (5011, 1764, 4)


## RPN 생성

In [13]:
class RPN(tf.keras.Model):
  def __init__(self, initializer, regularizer, shared_convNet):
    super(RPN, self).__init__(name='rpn')

    self.anchors = anchors
    self.anchor_size = anchor_size
    self.anchor_aspect_ratio = anchor_aspect_ratio
    self.Optimizers = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9)

    # 공용 레이어
    self.conv1_1 = SharedConvNet.layers[0]
    self.conv1_2 = SharedConvNet.layers[1]
    self.pooling_1 = SharedConvNet.layers[2]

    self.conv2_1 = SharedConvNet.layers[3]
    self.conv2_2 = SharedConvNet.layers[4]
    self.pooling_2 = SharedConvNet.layers[5]

    self.conv3_1 = SharedConvNet.layers[6]
    self.conv3_2 = SharedConvNet.layers[7]
    self.conv3_3 = SharedConvNet.layers[8]
    self.pooling_3 = SharedConvNet.layers[9]

    self.conv4_1 = SharedConvNet.layers[10]
    self.conv4_2 = SharedConvNet.layers[11]
    self.conv4_3 = SharedConvNet.layers[12]
    self.pooling_4 = SharedConvNet.layers[13]

    self.conv5_1 = SharedConvNet.layers[14]
    self.conv5_2 = SharedConvNet.layers[15]
    self.conv5_3 = SharedConvNet.layers[16]
    
    # RPN만의 레이어
    self.intermediate_layer = tf.keras.layers.Conv2D(512, (3, 3), padding = 'SAME' , activation = 'relu', name = "intermediate_layer", dtype='float32')
    self.cls_Layer = tf.keras.layers.Conv2D(18, (1, 1), kernel_initializer=initializer, padding = 'SAME' ,kernel_regularizer = regularizer, name = "output_1", dtype='float32')
    self.reg_layer = tf.keras.layers.Conv2D(36, (1, 1), kernel_initializer=initializer, padding = 'SAME' ,kernel_regularizer = regularizer, name = "output_2", dtype='float32')
    
  def call(self, inputs):
    # 정방향 연산
    # inputs = np.array(inputs)
    output = self.conv1_1(inputs)
    output = self.conv1_2(output)
    output = self.pooling_1(output)

    output = self.conv2_1(output)
    output = self.conv2_2(output)
    output = self.pooling_2(output)

    output = self.conv3_1(output)
    output = self.conv3_2(output)
    output = self.conv3_3(output)
    output = self.pooling_3(output)

    output = self.conv4_1(output)
    output = self.conv4_2(output)
    output = self.conv4_3(output)
    output = self.pooling_4(output)

    output = self.conv5_1(output)
    output = self.conv5_2(output)
    shared_output = self.conv5_3(output)
    # RPN
    shared_output = self.intermediate_layer(shared_output)
    cls_Layer_output = self.cls_Layer(shared_output)
    reg_Layer_output = self.reg_layer(shared_output) 

    return cls_Layer_output, reg_Layer_output # (1,14,14,18), (1,14,14,36) 반환

  def get_minibatch_index(self, cls_layer_output): # 라벨이니까 (1764,2) 넘파이 온다

    index_list = np.array([])
      
    index_list = np.zeros(1764) # 각 앵커가 미니배치 뽑혔나 안뽑혔나
    index_pos = np.array([])
    index_neg = np.array([])
    # cls_layer_output을 보고 긍정, 부정 앵커 분류. 그렇게 데이터셋을 구성함
    for i in range(0, 1764):
        if cls_layer_output[i][0] == 1.0 : # positive anchor
            index_pos = np.append(index_pos, i)
        elif cls_layer_output[i][0] == 0.0 : # negative anchor
            index_neg = np.append(index_neg, i)

    max_for = min([128, len(index_pos)])
    ran_list = random.sample(range(0, len(index_pos)), max_for)

    for i in range(0, len(ran_list)) :
        index = int(index_pos[ran_list[i]])
        index_list[index] = 1

    ran_list = random.sample(range(0, len(index_neg)), 256 - max_for) # 랜덤성 증가?를 위해 또다시 난수 생성
    for i in range(0, len(ran_list)) :
        index = int(index_neg[ran_list[i]])
        index_list[index] = 1

    return index_list # (1764,1) <- 1,0으로 이루어진 boolean 넘파이 배열

  # multi task loss
  def multi_task_loss(self ,image ,cls_layer_output_label, reg_layer_output_label):

    cls_layer_output, reg_layer_output = self.call(image) # 텐서 반환(상관성 존재). (1,14,14,18), (1,14,14,36)
    minibatch_index_list = self.get_minibatch_index(cls_layer_output_label) # 미니배치 인덱스 휙득

    # 모든 cls output을 softmax
    cls_layer_output = tf.reshape(cls_layer_output[0], [1764,2])
    cls_activation = tf.nn.softmax(cls_layer_output) # 잘됨

    # reg output는 앵커의 변화값임
    anchor_Use = np.reshape(anchors, (-1,4))
    anchor_tensor = tf.convert_to_tensor(anchor_Use, dtype=tf.float32)

    reg_layer_output = tf.reshape(reg_layer_output[0], [1764,4])
    reg_layer_output = tf.math.add(reg_layer_output, anchor_tensor)

    # label은 (1764,2)와 (1764,4)임
    tensor_cls_label = tf.convert_to_tensor(cls_layer_output_label, dtype=tf.float32)
    tensor_reg_label = tf.convert_to_tensor(reg_layer_output_label, dtype=tf.float32)

    
    # loss 계산(Loss 텐서에서 미니배치에 해당되는 애들만 걸러내야함)
    print("loss 계산")
    Cls_Loss = tf.nn.softmax_cross_entropy_with_logits(labels=tensor_cls_label, logits=cls_activation) # (1764,1) 텐서

    ## 여기까지 이상 무

    filter_x = tf.Variable([[1.0],[0.0],[0.0], [0.0]])
    filter_y = tf.Variable([[0.0],[1.0],[0.0], [0.0]])
    filter_w = tf.Variable([[0.0],[0.0],[1.0], [0.0]])
    filter_h = tf.Variable([[0.0],[0.0],[0.0], [1.0]])

    print("reg loss 계산")
    # 여기서 터짐
    x = tf.matmul(reg_layer_output,filter_x)
    y = tf.matmul(reg_layer_output,filter_y)
    w = tf.matmul(reg_layer_output,filter_w)
    h = tf.matmul(reg_layer_output,filter_h)

    x_a = tf.matmul(anchor_tensor,filter_x)
    y_a = tf.matmul(anchor_tensor,filter_y)
    w_a = tf.matmul(anchor_tensor,filter_w)
    h_a = tf.matmul(anchor_tensor,filter_h)

    x_star = tf.matmul(tensor_reg_label,filter_x)
    y_star = tf.matmul(tensor_reg_label,filter_y)
    w_star = tf.matmul(tensor_reg_label,filter_w)
    h_star = tf.matmul(tensor_reg_label,filter_h)

    denominator = tf.log(tf.constant(10, dtype=numerator.dtype)) # 텐서 로그는 ln밖에 없어서 ln10을 구한 뒤 나누는 방식으로 log10을 구한다

    t_x = tf.divide(tf.subtract(x, x_a), w_a)
    t_y = tf.divide(tf.subtract(y, y_a), h_a)
    t_w = tf.devide(tf.log(tf.divide(w, w_a)), denominator)
    t_h = tf.devide(tf.log(tf.divide(h, h_a)), denominator)

    t_x_star = tf.divide(tf.subtract(x_star, x_a), w_a)
    t_y_star = tf.divide(tf.subtract(y_star, y_a), h_a)
    t_w_star = tf.devide(tf.log(tf.divide(w_star, w_a)), denominator)
    t_h_star = tf.devide(tf.log(tf.divide(h_star, h_a)), denominator)

    Reg_Loss = tf.add_n(tf.compat.v1.losses.huber_loss(t_x_star, t_x), tf.compat.v1.losses.huber_loss(t_y_star, t_y), tf.compat.v1.losses.huber_loss(t_w_star, t_w), tf.compat.v1.losses.huber_loss(t_h_star, t_h))   

    # 미니배치에 해당되는 애들만 0이 아닌 값으로 만들기

    minibatch_index_list_cls = np.zeros((1764,2))
    minibatch_index_list_reg = np.zeros((1764,4))
    for i in range(0, len(minibatch_index_list)) :
      if minibatch_index_list[i] == 1:
        minibatch_index_list_cls[i][0] = 1
        minibatch_index_list_cls[i][1] = 1

        minibatch_index_list_reg[i][0] = 1
        minibatch_index_list_reg[i][1] = 1
        minibatch_index_list_reg[i][2] = 1
        minibatch_index_list_reg[i][3] = 1

    # (1764,2)와 (1764,4)의 텐서로 변환
    minibatch_index_list_cls = tf.convert_to_tensor(minibatch_index_list_cls, dtype=tf.float32)
    minibatch_index_list_reg = tf.convert_to_tensor(minibatch_index_list_reg, dtype=tf.float32)

    # 원소끼리 곱함 -> 미니배치 원소는 곱하기 1이 되고 나머지 원소는 곱하기 0이 되니까 미니배치 부분만 값이 남게 된다. 
    minibatch_cls = tf.multiply(minibatch_index_list_cls, Cls_Loss)
    minibatch_reg = tf.multiply(minibatch_index_list_reg, Reg_Loss)


    N_reg = tf.constant([1764.0])
    N_cls = tf.constant([10.0/256.0]) # lambda도 곱한 값

    loss = tf.add(tf.multiply(N_reg, tf.reduce_sum(minibatch_cls)), tf.multiply(N_cls, tf.reduce_sum(minibatch_reg)))
    
    return loss

    # get gradients
  def get_grad(self, Loss, training_step, cls_reg_boolean): # cls_reg_boolean = 0이면 cls, cls_reg_boolean = 1 이면 reg
    with tf.GradientTape() as tape:
      if training_step == 1 : # 처음에는 conv3_1까지만 훈련
        tape.watch(self.conv3_1.variables)
        tape.watch(self.conv3_2.variables)
        tape.watch(self.conv3_3.variables)
        tape.watch(self.conv4_1.variables)
        tape.watch(self.conv4_2.variables)
        tape.watch(self.conv4_3.variables)
        tape.watch(self.conv5_1.variables)
        tape.watch(self.conv5_2.variables)
        tape.watch(self.conv5_3.variables)
        tape.watch(self.intermediate_layer.variables)

        if cls_reg_boolean == 0:
          tape.watch(self.cls_Layer.variables)
        else:
          tape.watch(self.reg_layer.variables)

      elif training_step == 3:
        shared_output = self.intermediate_layer(shared_output)
        if cls_reg_boolean == 0:
          tape.watch(self.cls_Layer.variables)
        else:
          tape.watch(self.reg_layer.variables)

      g = 0
      if training_step == 1 :
        if cls_reg_boolean == 0:
          g = tape.gradient(Loss, [self.conv3_1.variables[0], self.conv3_1.variables[1], self.conv3_2.variables[0],self.conv3_2.variables[1], self.conv3_3.variables[0],self.conv3_3.variables[1], self.conv4_1.variables[0],self.conv4_1.variables[1], self.conv4_2.variables[0],self.conv4_2.variables[1], self.conv4_3.variables[0],self.conv4_3.variables[1], self.conv5_1.variables[0],self.conv5_2.variables[1], self.conv5_3.variables[0],self.conv5_3.variables[1], self.intermediate_layer.variables[0],self.intermediate_layer.variables[1], self.cls_Layer.variables[0],self.cls_Layer.variables[1]])
        else:
          g = tape.gradient(Loss, [self.conv3_1.variables[0], self.conv3_1.variables[1], self.conv3_2.variables[0],self.conv3_2.variables[1], self.conv3_3.variables[0],self.conv3_3.variables[1], self.conv4_1.variables[0],self.conv4_1.variables[1], self.conv4_2.variables[0],self.conv4_2.variables[1], self.conv4_3.variables[0],self.conv4_3.variables[1], self.conv5_1.variables[0],self.conv5_2.variables[1], self.conv5_3.variables[0],self.conv5_3.variables[1], self.intermediate_layer.variables[0],self.intermediate_layer.variables[1], self.reg_layer.variables[0],self.reg_layer.variables[1]])

      elif training_step == 3:
        if cls_reg_boolean == 0:
          g = tape.gradient(Loss, [self.intermediate_layer.variables[0],self.intermediate_layer.variables[1], self.cls_Layer.variables[0],self.cls_Layer.variables[1]])
        else:
          g = tape.gradient(Loss, [self.intermediate_layer.variables[0],self.intermediate_layer.variables[1], self.reg_layer.variables[0],self.reg_layer.variables[1]])

    return g
    
  def App_Gradient(self, Loss, training_step) :
    if training_step == 1:
      g_cls = self.get_grad(Loss, training_step, 0)
      self.Optimizers.apply_gradients(zip(g_cls, [self.conv3_1.variables[0], self.conv3_1.variables[1], self.conv3_2.variables[0],self.conv3_2.variables[1], self.conv3_3.variables[0],self.conv3_3.variables[1], self.conv4_1.variables[0],self.conv4_1.variables[1], self.conv4_2.variables[0],self.conv4_2.variables[1], self.conv4_3.variables[0],self.conv4_3.variables[1], self.conv5_1.variables[0],self.conv5_2.variables[1], self.conv5_3.variables[0],self.conv5_3.variables[1], self.intermediate_layer.variables[0],self.intermediate_layer.variables[1], self.cls_Layer.variables[0],self.cls_Layer.variables[1]]))

      g_reg = self.get_grad(Loss, training_step, 1)
      self.Optimizers.apply_gradients(zip(g_reg, [self.conv3_1.variables[0], self.conv3_1.variables[1], self.conv3_2.variables[0],self.conv3_2.variables[1], self.conv3_3.variables[0],self.conv3_3.variables[1], self.conv4_1.variables[0],self.conv4_1.variables[1], self.conv4_2.variables[0],self.conv4_2.variables[1], self.conv4_3.variables[0],self.conv4_3.variables[1], self.conv5_1.variables[0],self.conv5_2.variables[1], self.conv5_3.variables[0],self.conv5_3.variables[1], self.intermediate_layer.variables[0],self.intermediate_layer.variables[1], self.reg_layer.variables[0],self.reg_layer.variables[1]]))

    if training_step == 3:

      g_cls = self.get_grad(Loss, training_step, 0)
      self.Optimizers.apply_gradients(zip(g_cls, [self.intermediate_layer.variables[0],self.intermediate_layer.variables[1], self.cls_Layer.variables[0],self.cls_Layer.variables[1]]))

      g_reg = self.get_grad(Loss, training_step, 1)
      self.Optimizers.apply_gradients(zip(g_reg, [self.intermediate_layer.variables[0],self.intermediate_layer.variables[1], self.reg_layer.variables[0],self.reg_layer.variables[1]]))

  # perform gradient descent

  
  def Training_model(self, image_list, cls_layer_ouptut_label_list, reg_layer_ouptut_label_list, training_step):
    Loss_acc = 0
    for i in tqdm(range(0, len(image_list)), desc = "training"):

      image = np.expand_dims(image_list[i], axis = 0) # (1,224,224,3)으로 제작
      Loss = self.multi_task_loss(image, cls_layer_ouptut_label_list[i], reg_layer_ouptut_label_list[i])
      Loss_acc = Loss_acc + Loss
      if i % 19 == 0:
        Loss_acc = Loss_acc / 20
        self.App_Gradient(Loss, training_step)
        Loss_acc = 0


In [14]:
x = tf.Variable([[0,0,0,0],[0,1,0,0],[0,0,1,0], [0,0,0,0]])
y = tf.Variable([[2],[4],[7], [6]])
z = tf.matmul(x,y)

In [17]:
z

<tf.Tensor: id=17, shape=(4, 1), dtype=int32, numpy=
array([[0],
       [4],
       [7],
       [0]], dtype=int32)>

In [19]:
x = tf.Variable([[3,1]])
one = tf.one_hot(x, 4)

In [20]:
x

<tf.Variable 'Variable:0' shape=(1, 2) dtype=int32, numpy=array([[3, 1]], dtype=int32)>

## RPN 훈련

## RPN 모델 생성

In [15]:
max_num = len(tf.keras.applications.VGG16(weights='imagenet', include_top=False,  input_shape=(224, 224, 3)).layers) # 레이어 최대 개수

SharedConvNet = tf.keras.models.Sequential()
for i in range(0, max_num-1):
    SharedConvNet.add(tf.keras.applications.VGG16(weights='imagenet', include_top=False,  input_shape=(224, 224, 3)).layers[i])

initializer = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.01, seed=None)
regularizer = tf.keras.regularizers.l2(0.0005)

for layer in SharedConvNet.layers:
    # 'kernel_regularizer' 속성이 있는 인스턴스를 찾아 regularizer를 추가
    if hasattr(layer, 'kernel_regularizer'):
        setattr(layer, 'kernel_regularizer', regularizer)

RPN_Model = RPN(initializer, regularizer, SharedConvNet)

## RPN 훈련

In [16]:
# RPN_Model.Training_model(image_list, cls_layer_label_list, reg_layer_label_list, 1)

training:   0%|          | 0/5011 [00:00<?, ?it/s]

### RPN에서 값 얻어내기

In [16]:
def Process_output(cls_layer_output, reg_layer_output, anchor_size, anchor_aspect_ratio):

    for y in range(0, 14):
        for x in range(0, 14):
            for i in range(0, len(anchor_size)):
                for j in range(0, len(anchor_aspect_ratio)):
                    # 오브젝트다 vs 아니다 2개 항목에 대한 스코어 저장(각 앵커당)
                    index = 3 * i + j
                    object_score_exp = np.exp(cls_layer_output[y][x][2 * index]) # 출력값이 작아서 1e32를 구현하다
                    non_object_score_exp = np.exp(cls_layer_output[y][x][2 * index + 1]) 

                    exp_sum = object_score_exp + non_object_score_exp
                    score = np.array([object_score_exp / exp_sum, non_object_score_exp / exp_sum])
                    cls_layer_output[y][x][2*(3*i+j)] = score[0]
                    cls_layer_output[y][x][2*(3*i+j) + 1] = score[1]
                        
                    center_x = 8 + 16 * x 
                    center_y = 8 + 16 * y
                    w = anchor_size[i] * anchor_aspect_ratio[j][0]
                    h = anchor_size[i] * anchor_aspect_ratio[j][1]

                    # 원래 출력값은 각 위치 앵커에서 변화율로 사용한다. 
                    reg_layer_output[y][x][4*index] = center_x + reg_layer_output[y][x][4*index]*1e32
                    reg_layer_output[y][x][4*index + 1] = center_y + reg_layer_output[y][x][4*index + 1]*1e32
                    reg_layer_output[y][x][4*index + 2] = w + reg_layer_output[y][x][4*index + 2]*1e32
                    reg_layer_output[y][x][4*index + 3] = h + reg_layer_output[y][x][4*index + 3]*1e32

    cls_layer_output = np.reshape(cls_layer_output, (-1,2))
    reg_layer_output = np.reshape(reg_layer_output, (-1,4))

    return cls_layer_output, reg_layer_output


In [17]:
def Get_Output_RPN(RPN_Model, image_list) :

    cls_layer_output_list = np.array([])
    reg_layer_output_list = np.array([])

    anchor_size = [32, 64, 128] # 이미지 크기가 224*224라 32, 64, 128로 지정
    anchor_aspect_ratio = [[1,1],[1,0.5], [0.5,1]] # W*L기준

    for i in tqdm(range(0, len(image_list)), desc = "get_output"):
        cls_layer_output, reg_layer_output = RPN_Model(np.expand_dims(image_list[i], axis=0))

        cls_layer_output = cls_layer_output[0].numpy()
        reg_layer_output = reg_layer_output[0].numpy()

        # 가공하자
        cls_layer_output, reg_layer_output = Process_output(cls_layer_output, reg_layer_output, anchor_size, anchor_aspect_ratio)

        cls_layer_output_list = np.append(cls_layer_output_list, cls_layer_output)
        reg_layer_output_list = np.append(reg_layer_output_list, reg_layer_output)
    
    cls_layer_output_list = np.reshape(cls_layer_output_list, (-1,1764,2))
    reg_layer_output_list = np.reshape(reg_layer_output_list, (-1,1764,4))

    return cls_layer_output_list, reg_layer_output_list

In [18]:
cls_layer_output_list, reg_layer_output_list = Get_Output_RPN(RPN_Model, image_list) # 출력값 얻기

get_output: 100%|██████████| 5011/5011 [56:07<00:00,  1.49it/s]


#### 일단 RoI를 특성맵에 맞게 변환시켜서 표시해보자

## Fast R-CNN

### 입력값 : 공유 특성맵 + RoI
### 출력값 : 예측한 객체 종류, 객체의 좌표(r, c, h, w). (r,c)는 왼쪽위 좌표고 h,w는 너비, 높이

### 한 이미지에 대한 출력값 중 object score가 0.7 이상인 앵커들을 선별한다

In [65]:
def nms(cls_layer_output, reg_layer_output, reg_layer_label, index_classes): # 한 이미지에서 뽑아낸 RPN의 출력값 2개, 이미지에 있는 모든 앵커들이 참조한 Ground Truth Box, 그 Ground Truth Box의 Class
    nms_RoI_List = [] # 각 이미지마다 RoI 갯수가 다르다. 그래서 리스트로 저장한다. 리스트 안에 리스트 저장하는 방식으로 len = 5011인 리스트를 생성할거다 
    nms_GroundTruthBox_List = [] # NMS에 들어간 RoI가 참조한 Ground Truth Box 
    nms_Classes_List = []
    for i in range(0, len(cls_layer_output)) :
        if cls_layer_output[i][0] > 0.7 : # 해당 앵커의 object score가 0.7을 넘겼으면
            # 저장
            nms_RoI_List.append(reg_layer_output[i]) # RoI
            nms_GroundTruthBox_List.append(reg_layer_output_list[i]) # Ground Truth Box
            # 그 Box가 가리키는 클래스 인덱스
            nms_Classes_List.append(index_classes[i])



    return nms_RoI_List, nms_GroundTruthBox_List, nms_Classes_List # 선별한 애들을 반환

In [ ]:
def get_nms_list(cls_layer_output_list, reg_layer_output_list, reg_layer_label_list, index_classes_list) :
    # RPN output 2개, 이미지에서 각 앵커(1764)들이 어떤 Ground Truth Box보고 IoU 계산했는지, 이미지에서 각 앵커(1764)들이 어떤 '클래스'의 Ground Truth Box보고 IoU 계산했는지
    # Detector 훈련에 필요한 데이터를 얻는 곳이다
    
    NMS_RoIs_List = [] # 전체 입력 이미지의 RoI를 이미지별로 저장(리스트 안에 리스트)
    NMS_GroundTruthBoxes_List = []
    NMS_Classes_List = []

    for i in tqdm(range(0, len(cls_layer_output_list)), desc = "get_RoI"): # 5011개에 대한 nms 구한다
        nms_RoI_List, nms_GroundTruthBox_List, nms_Classes_List = nms(cls_layer_output_list[i], reg_layer_output_list[i], reg_layer_label_list[i], index_classes_list[i])
        # 각 이미지에서 RoI 얻은 정보를 담기
        NMS_RoIs_List.append(nms_RoI_List)
        NMS_GroundTruthBoxes_List.append(NMS_GroundTruthBoxes_List)
        NMS_Classes_List.append(nms_Classes_List)

    return NMS_RoIs_List, NMS_GroundTruthBoxes_List, NMS_Classes_List

In [63]:
def change_RoI(nms_RoI_List) :
    # RoI_list : (-1, 4) <- (x,y,w,h)
    RoI_list_forDetector = np.array([])
    for i in range(0, len(RoI_list)) :
        r = RoI_list[i][0] - (RoI_list[i][2]/2)
        c = RoI_list[i][1] - (RoI_list[i][3]/2)
        w = RoI_list[i][2]
        h = RoI_list[i][3]

        roi_forDetector = np.array([r,c,w,h])
        RoI_list_forDetector = np.append(RoI_list_forDetector, roi_forDetector)
    RoI_list_forDetector = np.reshape(RoI_list_forDetector, (-1,4)) 

    return RoI_list_forDetector

## Detector 훈련에 필요한 데이터셋 휙득

In [ ]:
NMS_RoIs_List, NMS_GroundTruthBoxes_List, NMS_Classes_List = get_nms_list(cls_layer_output_list, reg_layer_output_list, reg_layer_label_list, index_classes_list)

## Detector 모델 생성

### RoI Pooling Layer 제작

In [ ]:
RoI max pooling works by dividing the h × w RoI window into an H × W grid of sub-windows of approximate size h/H × w/W and then max-pooling the values in each sub-window into the corresponding output grid cell.

In [ ]:
# Pooling 작업을 RoI 지역에 대해 한다. 14*14*512를 7*7*512로
class RoiPoolingLayer(tf.keras.layers):
    def __init__(self, pool_size, num_rois, **kwargs):
        super(RoiPoolingLayer, self).__init__(name='RoI_Pooling_Layer')

        self.pool_size = pool_size # VGG16에서는 7*7
        self.num_rois = num_rois # RoI 개수
        
    def build(self, input_shape): # input shape로 (1,14,14,512)와 같이 받으니까 3번 원소 자리의 값인 512를 가져간다. 
        self.nb_channels = input_shape[0][3] # 채널 조정
        # 맨처음에 값을 입력받을 때 가중치를 초기화 할 수 있는데 얘가 그걸 담당한다. 
        # 레이어가 받은 입력 크기를 기준으로 나중에 변수를 만들 수 있다는 장점이 있다. 

    def compute_output_shape(self, input_shape): # If the layer has not been built, this method will call build on the layer. 
        return None, self.num_rois, self.pool_size, self.pool_size, self.nb_channels

    def call(self, image, RoI_List): # 정방향 연산
        assert(len(x) == 2)

        # (1,14,14,512)에서 (14,14,512)를 가져감
        img = x[0]

        # x[1] is roi with shape (num_rois,4) with ordering (x,y,w,h)
        rois = x[1]

        input_shape = tf.shape(img) # <tf.Tensor: shape=(14,14,512), dtype=int32, numpy=array([...], dtype=float32)>

        RoiPooling_outputs_List = [] # RoI 부근을 잘라낸 뒤 7*7로 만들어낸 것들을 여기에 모은다

        for roi_idx in range(0, self.num_rois): # 이미지 당 RoI 갯수만큼 for문 반복 -> RoI 갯수만큼 특성맵 얻으려고
            x = RoI_List[i][0]
            y = RoI_List[i][1]
            w = RoI_List[i][2]
            h = RoI_List[i][3]

            # Resized roi of the image to pooling size (7x7)
            Resize_featuremap = tf.image.resize(img[:, y:y+h, x:x+w, :], (self.pool_size, self.pool_size)) # RoI부분만 빼서 7*7로 리사이징 해준다. 원래 Pooling은 전체 범위에 대해 이 작업을 한다. 
            RoiPooling_outputs_List.append(Resize_featuremap) # 리스트에 추가

        Pooling_output = tf.concat(outputs, axis=0)

        Pooling_output = tf.reshape(final_output, (1, self.num_rois, self.pool_size, self.pool_size, self.nb_channels)) # output 양식이 (1,...)니까 (1, RoI개수, 7,7,512)와 같이 만들어준다. 
        Pooling_output = tf.keras.backend.permute_dimensions(final_output, (0, 1, 2, 3, 4))

        return Pooling_output
    
    def get_config(self): # 구성요소 반환
        config = {'pool_size': self.pool_size,
                  'num_rois': self.num_rois}
        base_config = super(RoiPoolingLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [ ]:
class Detector(tf.keras.Model):

  def __init__(self, initializer, regularizer, shared_convNet):
    super(Detector, self).__init__(name='Detector')
    # 레이어 
    # 공용 레이어(14*14*512)를 받아서 RoI Pooling Layer에 넣어 7*7*512를 만들고 FCs를 거쳐 두가지 Output을 생성한다. 

    # 클래스 분류 레이어와 박스 위치 회귀 레이어의 초기화를 다르게 해야한다. 
    Classify_layer_initializer = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.01, seed=None)
    Box_regression_layer_initializer = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.001, seed=None)

    # RoI Pooling : H*W(7*7)에 맞게 입력 특성맵을 pooling. RoI에 해당하는 영역을 7*7로 Pooling한다. 
    self.RoIPoolingLayer = RoiPoolingLayer(7, num_rois) # Pooling 이후 크기를 7*7*512로 만든다. 
    self.Flatten_layer = tf.keras.layers.Flatten() # 여기서 4096개가 되어야한다.
    self.Fully_Connected = tf.keras.layers.Dense(4096, activation='relu') # 별 말이 없으니 기본적으로 지정된 kernel_initializer를 사용하자
    self.Classify_layer = tf.keras.layers.Dense(21, activation='softmax', kernel_initializer = Classify_layer_initializer, name = "output_1")
    self.reg_layer = tf.keras.layers.Conv2D(84, activation= None, kernel_initializer = Box_regression_layer_initializer, name = "output_2")
    
  def call(self, input1, input2): # input으로 공용 컨볼루션 레이어 텐서(1,14,14,512)와 RoI를 받는다. RoI는 앞서 RPN에서 뽑아낸걸 쓴다. 
    
    output = self.RoIPoolingLayer(input1, input2) # input1 : (1,14,14,512), input2 : RoI -> 7*7 output
    output = self.Flatten_layer(output) # Flatten으로 한줄 세우기
    output = self.Fully_Connected(output) # FCs로 만들기
    # 객체 분류 레이어, 박스 회귀 레이어
    cls_output = Classify_layer_output = self.Classify_layer(output) # 각 객체에 
    reg_layer_output = self.reg_layer(output)

    return Classify_layer_output, reg_layer_output # 두 아웃풋을 내놓는다. 

  def get_grad(self, Loss, training_step, cls_reg_boolean):
  
  def App_Gradient(self, Loss, training_step) :

  
  def training_Detector():


## Loss 함수

## 모델 생성

In [ ]:
max_num = len(tf.keras.applications.VGG16(weights='imagenet', include_top=False,  input_shape=(224, 224, 3)).layers) # 레이어 최대 개수

SharedConvNet = tf.keras.models.Sequential()
for i in range(0, max_num-1):
    SharedConvNet.add(tf.keras.applications.VGG16(weights='imagenet', include_top=False,  input_shape=(224, 224, 3)).layers[i])

Detector_Model = Detector(SharedConvNet)
RPN_Model.run_eagerly = True # 모델 내부 함수에서 쉽게 연산할 수 있게 설정

In [ ]:
def training_Detector(model, , traning_step, EPOCH):

    losses = {'output_1' : 'categorical_crossentropy', 'output_2' : loss_detector_loc} # classify layer는 20 + 1가지 클래스에 대한 softmax, reg_layer는 따로 만든 loss함수

    model.compile(optimizer=Optimizer, loss=losses, run_eagerly=True)

    history = model.fit(np.asarray(image_list), [cls_layer_label_list, reg_layer_label_list], batch_size = 1, epochs = EPOCH)

    return model, history
